In [ ]:
#将16进制数据转换成实际电压/电流值

# 数据帧解析器
class DataFrameParser:
    def __init__(self):
        self.FRAME_HEADER = 0xAA  # 假设帧头为0xAA，需要根据实际定义调整
        self.FRAME_FOOTER = 0xBB  # 假设帧尾为0x55，需要根据实际定义调整
        
        # ADC参数 (16位ADC)
        self.ADC_RESOLUTION = 16
        self.ADC_MAX_VALUE = (1 << self.ADC_RESOLUTION) - 1  # 65535
        self.ADC_VREF = 5  # 参考电压，根据实际情况调整
        
        # AFE参数 (24位ADC)
        self.AFE_RESOLUTION = 24
        self.AFE_MAX_VALUE = (1 << self.AFE_RESOLUTION) - 1  # 16777215
        self.AFE_VREF = 8000  # 参考电压，根据实际情况调整
    
    def parse_hex_string(self, hex_string):
        """
        解析16进制字符串为字节数组
        输入格式支持: "AA 12 34 56 78 9A BC 55" 或 "AA123456789ABC55"
        """
        # 移除空格和其他分隔符
        clean_hex = hex_string.replace(' ', '').replace(',', '').replace('-', '')
        
        # 检查长度
        if len(clean_hex) != 16:  # 8字节 = 16个16进制字符
            raise ValueError(f"数据长度错误: 期望16个字符，实际{len(clean_hex)}个字符")
        
        # 转换为字节数组
        try:
            bytes_data = bytes.fromhex(clean_hex)
        except ValueError as e:
            raise ValueError(f"16进制格式错误: {e}")
        
        return bytes_data
    
    def parse_frame(self, hex_string):
        """
        解析8字节数据帧
        """
        try:
            data = self.parse_hex_string(hex_string)
            
            # 提取各字段
            header = data[0]
            adc_high = data[1]
            adc_low = data[2]
            afe_high = data[3]
            afe_medium = data[4]
            afe_low = data[5]
            checksum = data[6]
            footer = data[7]
            
            # 验证帧头帧尾
            header_valid = header == self.FRAME_HEADER
            footer_valid = footer == self.FRAME_FOOTER
            
            # 重构16位ADC数据
            adc_data = (adc_high << 8) | adc_low
            
            # 重构24位AFE数据
            afe_data = (afe_high << 16) | (afe_medium << 8) | afe_low
            
            # 计算期望的校验和（位与运算）
            expected_checksum = adc_high & adc_low & afe_high & afe_medium & afe_low
            checksum_valid = checksum == expected_checksum
            
            # 转换为电压值
            adc_voltage = (adc_data / self.ADC_MAX_VALUE) * self.ADC_VREF
            afe_voltage = ((afe_data / self.AFE_MAX_VALUE) /128)* self.AFE_VREF 
            
            return {
                'raw_data': hex_string,
                'parsed_bytes': [f"0x{b:02X}" for b in data],
                'header': f"0x{header:02X}",
                'header_valid': header_valid,
                'footer': f"0x{footer:02X}",
                'footer_valid': footer_valid,
                'adc_raw': adc_data,
                'adc_voltage': adc_voltage,
                'afe_raw': afe_data,
                'afe_voltage': afe_voltage,
                'checksum': f"0x{checksum:02X}",
                'expected_checksum': f"0x{expected_checksum:02X}",
                'checksum_valid': checksum_valid,
                'frame_valid': header_valid and footer_valid and checksum_valid
            }
            
        except Exception as e:
            return {'error': str(e)}
    
    def print_result(self, result):
        """
        格式化输出解析结果
        """
        if 'error' in result:
            print(f"❌ 解析错误: {result['error']}")
            return
        
        print("=" * 60)
        print(f"📊 数据帧解析结果")
        print("=" * 60)
        print(f"原始数据: {result['raw_data']}")
        print(f"字节数组: {' '.join(result['parsed_bytes'])}")
        print()
        
        print("🔍 帧结构验证:")
        print(f"  帧头: {result['header']} {'✅' if result['header_valid'] else '❌'}")
        print(f"  帧尾: {result['footer']} {'✅' if result['footer_valid'] else '❌'}")
        print(f"  校验和: {result['checksum']} (期望: {result['expected_checksum']}) {'✅' if result['checksum_valid'] else '❌'}")
        print(f"  整体有效性: {'✅ 有效' if result['frame_valid'] else '❌ 无效'}")
        print()
        
        print("⚡ 电压转换:")
        print(f"  16位ADC:")
        print(f"    原始值: {result['adc_raw']} (0x{result['adc_raw']:04X})")
        print(f"    电压值: {result['adc_voltage']:.6f} V")
        print(f"  24位AFE:")
        print(f"    原始值: {result['afe_raw']} (0x{result['afe_raw']:06X})")
        print(f"    电压值: {result['afe_voltage']:.6f} V")
        print("=" * 60)

# 创建解析器实例
parser = DataFrameParser()

In [7]:
# 交互式数据输入和解析
def interactive_parse():
    """
    交互式解析数据帧
    """
    print("🔧 ESP32数据帧解析器")
    print("请输入8字节16进制数据 (支持空格分隔):")
    print("示例格式: AA 12 34 56 78 9A BC 55 或 AA123456789ABC55")
    print("输入 'quit' 退出")
    print()
    
    while True:
        try:
            user_input = input("请输入数据: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("👋 再见!")
                break
            
            if not user_input:
                print("❌ 请输入有效数据")
                continue
            
            # 解析数据
            result = parser.parse_frame(user_input)
            parser.print_result(result)
            
            print("\n继续输入下一组数据，或输入 'quit' 退出")
            print("-" * 40)
            
        except KeyboardInterrupt:
            print("\n👋 程序已退出")
            break
        except Exception as e:
            print(f"❌ 未知错误: {e}")

# 测试数据示例
def test_examples():
    """
    使用示例数据测试解析器
    """
    print("🧪 测试示例数据:")
    
    # 示例1: 正常数据
    test_data1 = "AA 10 20 30 40 50 00 BB"  # 假设这是有效数据
    print(f"\n测试数据1: {test_data1}")
    result1 = parser.parse_frame(test_data1)
    parser.print_result(result1)
    
    # 示例2: 校验和错误的数据
    test_data2 = "AA 10 20 30 40 50 FF BB"  # 校验和错误
    print(f"\n测试数据2: {test_data2}")
    result2 = parser.parse_frame(test_data2)
    parser.print_result(result2)

# 运行测试
print("选择操作:")
print("1. 测试示例数据")
print("2. 交互式输入")
choice = input("请输入选择 (1/2): ").strip()

if choice == "1":
    test_examples()
elif choice == "2":
    interactive_parse()
else:
    print("启动交互式模式...")
    interactive_parse()

选择操作:
1. 测试示例数据
2. 交互式输入
🔧 ESP32数据帧解析器
请输入8字节16进制数据 (支持空格分隔):
示例格式: AA 12 34 56 78 9A BC 55 或 AA123456789ABC55
输入 'quit' 退出

🔧 ESP32数据帧解析器
请输入8字节16进制数据 (支持空格分隔):
示例格式: AA 12 34 56 78 9A BC 55 或 AA123456789ABC55
输入 'quit' 退出

📊 数据帧解析结果
原始数据: AA 42 02 1F 4E BF 02 BB
字节数组: 0xAA 0x42 0x02 0x1F 0x4E 0xBF 0x02 0xBB

🔍 帧结构验证:
  帧头: 0xAA ✅
  帧尾: 0xBB ✅
  校验和: 0x02 (期望: 0x02) ✅
  整体有效性: ✅ 有效

⚡ 电压转换:
  16位ADC:
    原始值: 16898 (0x4202)
    电压值: 1.289235 V
  24位AFE:
    原始值: 2051775 (0x1F4EBF)
    电压值: 7.643458 V

继续输入下一组数据，或输入 'quit' 退出
----------------------------------------
📊 数据帧解析结果
原始数据: AA 42 02 1F 4E BF 02 BB
字节数组: 0xAA 0x42 0x02 0x1F 0x4E 0xBF 0x02 0xBB

🔍 帧结构验证:
  帧头: 0xAA ✅
  帧尾: 0xBB ✅
  校验和: 0x02 (期望: 0x02) ✅
  整体有效性: ✅ 有效

⚡ 电压转换:
  16位ADC:
    原始值: 16898 (0x4202)
    电压值: 1.289235 V
  24位AFE:
    原始值: 2051775 (0x1F4EBF)
    电压值: 7.643458 V

继续输入下一组数据，或输入 'quit' 退出
----------------------------------------

👋 程序已退出

👋 程序已退出
